## Mục lục
### 1. Chuyển classical music dạng .mid sang dạng .wav
### 2. Đọc phổ âm thanh

### 1. Chuyển classical music dạng .mid sang dạng .wav

In [1]:
import pretty_midi
import soundfile as sf
import os
import librosa
import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt

In [2]:
os.makedirs("songs", exist_ok=True)
os.makedirs("songs/classical_mussic", exist_ok=True)
os.makedirs("songs/edm_mussic", exist_ok=True)
classsical_midi_files = [
    "classical_mussic/albeniz/alb_esp1.mid",
    "classical_mussic/bach/bach_846.mid",
    "classical_mussic/mozart/mz_311_3.mid",
    "classical_mussic/grieg/grieg_album.mid",
    "classical_mussic/brahms/br_im2.mid"
    ]
for file in classsical_midi_files:
    midi_data_classical = pretty_midi.PrettyMIDI(file)
    audio_data_classical = midi_data_classical.synthesize(fs=44100)
    sf.write("songs/classical_mussic"+file[file.rfind('/'):file.rfind(".")]+".wav", audio_data_classical, 44100)

### 2. Đọc phổ âm thanh

In [3]:
def phan_tich_va_luu_pho_am_thanh(
    file_path: str,
    save_dir: str,
    offset_seconds: float = 0.0,
    duration_seconds: float = 5.0,
    plot_title_prefix: str = "Phổ âm thanh"
):
    """
    Phân tích phổ Fourier của một đoạn âm thanh từ file và lưu biểu đồ.

    Args:
        file_path (str): Đường dẫn đến file âm thanh (WAV, MP3, v.v.).
        save_dir (str): Thư mục để lưu biểu đồ phổ âm thanh.
        offset_seconds (float): Thời điểm bắt đầu (tính bằng giây) của đoạn âm thanh cần phân tích.
                                Mặc định là 0.0 (bắt đầu file).
        duration_seconds (float): Độ dài (tính bằng giây) của đoạn âm thanh cần phân tích.
                                  Mặc định là 5.0 giây.
        plot_title_prefix (str): Tiền tố cho tiêu đề của biểu đồ.
    """
    try:
        # 1. Tải đoạn âm thanh với sampling rate gốc
        audio_segment, samplerate = librosa.load(
            file_path,
            sr=None,
            offset=offset_seconds,
            duration=duration_seconds
        )

        # Kiểm tra nếu đoạn âm thanh không hợp lệ hoặc rỗng
        if audio_segment.size == 0:
            print(f"Lỗi: Không thể tải hoặc đoạn âm thanh rỗng từ '{file_path}' "
                  f"với offset={offset_seconds}s, duration={duration_seconds}s.")
            return

        # 2. Thực hiện biến đổi Fourier (FFT)
        N = len(audio_segment)
        yf = fft(audio_segment)                      # Tín hiệu sau FFT (dạng phức)
        xf = fftfreq(N, 1 / samplerate)              # Trục tần số

        # Chỉ lấy phần tần số dương
        positive_xf = xf[:N//2]
        amplitude_dB = 20 * np.log10(np.abs(yf[:N//2]) + 1e-9)  # Biên độ theo đơn vị dB

        # 3. Tạo biểu đồ phổ tần số
        plt.figure(figsize=(10, 5))
        plt.plot(positive_xf, amplitude_dB)
        
        title = f"{plot_title_prefix} - {os.path.basename(file_path)}\n" \
                f"(Từ {offset_seconds:.1f}s - {offset_seconds + duration_seconds:.1f}s, SR: {samplerate} Hz)"
        plt.title(title)
        plt.xlabel("Tần số (Hz)")
        plt.ylabel("Biên độ (dB)")
        plt.grid(True)
        plt.xlim(0, min(20000, samplerate / 2))  # Giới hạn hiển thị tới 20kHz hoặc Nyquist
        plt.tight_layout()

        # 4. Lưu biểu đồ
        os.makedirs(save_dir, exist_ok=True)
        base_filename = os.path.splitext(os.path.basename(file_path))[0]
        save_filename = f"spectrum_{base_filename}_offset{offset_seconds:.0f}s_dur{duration_seconds:.0f}s.png"
        plot_save_path = os.path.join(save_dir, save_filename)

        plt.savefig(plot_save_path)
        plt.close()  # Giải phóng bộ nhớ sau khi vẽ
        print(f"Đã lưu biểu đồ phổ âm thanh tại: {plot_save_path}")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file âm thanh tại '{file_path}'")
    except Exception as e:
        print(f"Lỗi không xác định khi xử lý file '{file_path}': {e}")
        if plt.get_fignums():
            plt.close()

In [4]:
classic_music_file = os.listdir("songs/classical_mussic")
edm_music_file = os.listdir("songs/edm_mussic")
for file in classic_music_file:
    phan_tich_va_luu_pho_am_thanh(
        file_path=os.path.join("songs/classical_mussic", file),
        save_dir="bieu_do_pho_am_thanh/classical_mussic",
        offset_seconds=30.0,
        duration_seconds=3.0,
        plot_title_prefix="Phổ Nhạc Cổ Điển"
    )
for file in edm_music_file:
    phan_tich_va_luu_pho_am_thanh(
        file_path=os.path.join("songs/edm_mussic", file),
        save_dir="bieu_do_pho_am_thanh/edm_mussic",
        offset_seconds=115.0,
        duration_seconds=3.0,
        plot_title_prefix="Phổ Nhạc Điện Tử"
    )
print("Đã hoàn thành phân tích và lưu phổ âm thanh cho các file nhạc.")

Đã lưu biểu đồ phổ âm thanh tại: bieu_do_pho_am_thanh/classical_mussic/spectrum_mz_311_3_offset30s_dur3s.png
Đã lưu biểu đồ phổ âm thanh tại: bieu_do_pho_am_thanh/classical_mussic/spectrum_alb_esp1_offset30s_dur3s.png
Đã lưu biểu đồ phổ âm thanh tại: bieu_do_pho_am_thanh/classical_mussic/spectrum_bach_846_offset30s_dur3s.png
Đã lưu biểu đồ phổ âm thanh tại: bieu_do_pho_am_thanh/classical_mussic/spectrum_grieg_album_offset30s_dur3s.png
Đã lưu biểu đồ phổ âm thanh tại: bieu_do_pho_am_thanh/classical_mussic/spectrum_br_im2_offset30s_dur3s.png
Đã lưu biểu đồ phổ âm thanh tại: bieu_do_pho_am_thanh/edm_mussic/spectrum_animals_offset115s_dur3s.png
Đã lưu biểu đồ phổ âm thanh tại: bieu_do_pho_am_thanh/edm_mussic/spectrum_the_spectre_offset115s_dur3s.png
Đã lưu biểu đồ phổ âm thanh tại: bieu_do_pho_am_thanh/edm_mussic/spectrum_move_your_body_offset115s_dur3s.png
Đã lưu biểu đồ phổ âm thanh tại: bieu_do_pho_am_thanh/edm_mussic/spectrum_never_be_alone_offset115s_dur3s.png
Đã lưu biểu đồ phổ âm th